<a href="https://colab.research.google.com/github/jutrera/InnoSoft-2018/blob/master/InnoSoft_2018_Deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InnoSoft 2018 - Deep Learning
En este ejercicio vamos a entrenar un modelo de Deep Learning. Keras dispone de modelos ya preentrenados. En este caso entrenaremos una red ResNet

## Preparamos el entorno

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [0]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
print('Files in Drive:')
!ls drive

In [0]:
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## API de Kaggle
Para acceder a los dataset, vamos a valernos de la API de Kaggle, el cual nos va a permitir descargarnos un conjunto de datos ya existente.

En primer lugar, instalamos la API

In [0]:
!pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Guardamos el archivo de configuración en **root**.

In [0]:
!mkdir '/root/.kaggle'
!ls /content/.kaggle
!cp "/content/.kaggle/kaggle.json" "/root/.kaggle/kaggle.json"

!ls /root/.kaggle

Vamos a hacer un listado de los dataset propios

In [0]:

!kaggle datasets list -m

Y nos descargamos el dataset elegido

In [0]:
!kaggle datasets download --unzip 'jutrera/stanford-car-dataset-by-classes-folder'


Veamos la estruvtura de directorios

In [0]:
!ls "./"

Descomprimiremos las imágenes.

In [0]:
import zipfile

zf=zipfile.ZipFile("./car_data.zip", "r")
for i in zf.namelist():
    zf.extract(i, path="./")
    
!ls './car_data'

Vamos a ver una de las imágenes.

In [0]:
path_base = './'

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_base + '/car_data/train/Volvo XC90 SUV 2007') ]
from PIL import Image

image = Image.open(path_base + '/car_data/train/Volvo XC90 SUV 2007/' + onlyfiles[0])

imgplot = plt.imshow(image)
plt.show()

##Training data

Empezamos importando librerías.

In [0]:
import numpy as np
from scipy import misc
from PIL import Image
import glob
import matplotlib.pyplot as plt
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from IPython.display import SVG
import cv2
import seaborn as sn
import pandas as pd
import pickle
import time

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras_metrics import precision, recall
from keras import backend as K

Inicializamos y definimos valores para el entrenamiento (numero de epochs, tamaño del batch, etc).

In [0]:
K.set_learning_phase(1)
K.set_image_data_format('channels_last')

In [0]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 8041
epochs = 2
batch_size = 32
n_classes = 196

###ImageDataGenerator
La clase ImageDataGenerator de Keras nos permitirá trabajar con grandes cantidades de imágenes de tamaño mayor. Este generator nos permitirá ir cargando en batches definidos las imágenes y así no provocar fallos de memoria por volúmenes de dataset excesivamente altos. Además, nos permite incluir modificaciones en las imágenes tales como Zoom, Pan, Rotaciones, etc, lo que permite aumentar el tamaño de los datasets de imágenes y *emular* los fallos de las fotos (que no sean *perfectas*).

In [0]:
train_data_dir = path_base + '/car_data/train'
validation_data_dir = path_base + '/car_data/test'

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    rotation_range = 5,
    width_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical')

### Entrenando una red profunda
Con esto, podemos definir el modelo. En este caso vamos a entrenar una [DenseNet](https://arxiv.org/abs/1512.03385). Esta red se basa en que las salidas o resultados de cada **bloque denso** pasa a ser parte de la entrada de todos los bloques posteriores.

![Diagrama de arquitectura DenseNet. Se representa un bloque denso de 5 capas con una tasa de crecimiento k = 4. Cada capa toma como entrada todos los valores de características anteriores](http://jesusutrera.com/articles/img/densenet01.jpg)

Diagrama de arquitectura DenseNet. Se representa un bloque denso de 5 capas con una tasa de crecimiento k = 4. Cada capa toma como entrada todos los valores de características anteriores

![Red neuronal de tres bloques densos. Las capas entre dos bloques adyacentes hacer referencia a una capa de transición cambiando el tamaño del mapa de características mediante convolución y pooling](http://jesusutrera.com/articles/img/densenet02.jpeg)

Red neuronal de tres bloques densos. Las capas entre dos bloques adyacentes hacer referencia a una capa de transición cambiando el tamaño del mapa de características mediante convolución y pooling.

El siguiente ejercicio consiste en:
1.  definir la red densa 121:
  *  Definir el **input_shape=(None, None, 3)**,
  *  cargar los pesos entrenados de **Imagenet**, 
  *  no incluir una capa densa por defecto (lo haremos nosotros) y
  * definir el pooling **Average**.
2.  Añadir a la red densa una capa de 500 nodos con activación ReLu.

In [0]:
from keras.applications import densenet

def build_model():
  #Code here 1
  base_model = densenet....
  #End Code
  
  #Si se desea dejar los pesos originales sin actualizar, poner False (esto haría que solo se entrene las redes regulares finales)
  for layer in base_model.layers:
    layer.trainable = True

  #Asociamos la salida de la DenseNet y añadimos una capa densa de 1000 nodos
  x = base_model.output
  x = Dense(1000)(x)
  x = Activation('relu')(x)
  
  #Code here 2 - Añadir la capa densa de 500 nodos con activación ReLu
  
  #End Code
  
  #Definimos la salida al número de clases
  predictions = Dense(n_classes, activation='softmax')(x)
  
  #y generamos el modelo
  model = Model(inputs=base_model.input, outputs=predictions)

  return model

La métrica RMSE (raíz cuadrada de la media de los errores cuadráticos) tiene la característica de que los valores tienen las mismas unidades que los valores de entrenamiento. Vamos a definir nuestra propia métrica. 

In [0]:
def rmse(y_true, y_pred):
	return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

Debido a que el entrenamiento de una red profunda es largo y costoso, vamos a obtener los pesos de un entrenamiento previo. Si no, no daría tiempo a realizar este taller ;).

Una vez cargados, compilamos el modelo.

In [0]:
model = build_model()
path_to_load = "drive/InnoSoft 2018/car_saved_01.h5"
model.load_weights(path_to_load)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse', rmse])

Hacemos un resumen del modelo construido. Podemos ver que, aunque hemos creado una red muy profunda, solo tenemos algo más de 8 millones de parámetros a entrenar. Si usaramos una red profunda más antigua, estaríamos hablando de 20 millones aproximadamente :(.

In [0]:
model.summary()

Definimos los callback siguientes:
* **Early stopping**: Si el modelo después de varias vueltas no mejora la función de perdida, paramos el entrenamiento.
* **Reduce LR on plateau**: Si después de un n´mero determinado de vueltas, la función de pérdida no mejora, probamos a reducir el valor de **learning rate**. esto suele hacer que el entrenamiento mejore en muchos casos.

In [0]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)

In [0]:
callbacks_list = [early_stop, reduce_lr]

Por último, entrenamos y vemos los resultados iniciales en la gráfica

In [0]:
model_history = model.fit_generator(
    train_generator,
    epochs=epochs,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)

In [0]:
plt.figure(0)
plt.plot(model_history.history['acc'],'r')
plt.plot(model_history.history['val_acc'],'g')
plt.xticks(np.arange(0, epochs, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(model_history.history['loss'],'r')
plt.plot(model_history.history['val_loss'],'g')
plt.xticks(np.arange(0, epochs, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.figure(2)
plt.plot(model_history.history['mean_squared_error'],'r')
plt.plot(model_history.history['val_mean_squared_error'],'g')
plt.xticks(np.arange(0, epochs, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training MSE vs Validation MSE")
plt.legend(['train','validation'])

plt.figure(3)
plt.plot(model_history.history['rmse'],'r')
plt.plot(model_history.history['val_rmse'],'g')
plt.xticks(np.arange(0, epochs, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("RMSE")
plt.title("Training RMSE vs Validation RMSE")
plt.legend(['train','validation'])
 
plt.show()

Si deseamos guardar el modelo entrenado, hacemos lo siguiente:

In [0]:
model.save('drive/InnoSoft 2018/car_nosize.h5')

Finalmente, evaluamos el modelo y obtenemos la matriz de confusión y el informe

In [0]:
model.evaluate_generator(validation_generator, steps=(nb_validation_samples // batch_size)+1, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

In [0]:
pred = model.predict_generator(validation_generator, steps=(nb_validation_samples // batch_size) + 1, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
predicted = np.argmax(pred, axis=1)
print(predicted)
print(len(predicted))

In [0]:
import cv2
import seaborn as sn
import pandas as pd
import pickle

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from keras.utils import to_categorical

my_test = to_categorical(validation_generator.classes)

Generamos el array de nombres de clases

In [0]:
import csv

class_names = []
with open(path_base + '/names.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile, delimiter=';')
    for row in csvReader:
        class_names.append(row[0])

In [0]:
print('Confusion Matrix')
cm = confusion_matrix(validation_generator.classes, np.argmax(pred, axis=1))
plt.figure(figsize = (30,20))
sn.set(font_scale=1.4) #for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
plt.show()
print()
print('Classification Report')
print(classification_report(validation_generator.classes, predicted, target_names=class_names))

In [0]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle

n_classes = 196

from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(my_test[:, i], pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(my_test.ravel(), pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(10), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

Hecho esto, podemos ver los resultados predichos por el modelo en algunas de las imágenes de validación

In [0]:
def predict_one(model):
    image_batch, classes_batch = next(validation_generator)
    predicted_batch = model.predict(image_batch)
    for k in range(0,image_batch.shape[0]):
      image = image_batch[k]
      pred = predicted_batch[k]
      the_pred = np.argmax(pred)
      predicted = class_names[the_pred]
      val_pred = max(pred)
      the_class = np.argmax(classes_batch[k])
      value = class_names[np.argmax(classes_batch[k])]
      plt.figure(k)
      eso = (the_pred == the_class)
      plt.title(str(eso) + ' - class: ' + value + ' - ' + 'predicted: ' + predicted + '[' + str(val_pred) + ']')
      plt.imshow(image)

predict_one(model)